In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
def load_generation_data(selected_files=None):
    # data/generation 디렉토리 경로
    data_dir = '/Users/jangseohyun/Documents/workspace/symply/DER/data/generation'
    
    # 디렉토리 내의 모든 CSV 파일 목록 가져오기
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])
    
    # 불러올 파일 선택
    files_to_load = all_files if selected_files is None else selected_files
    
    # 선택된 파일들 불러오기
    data_frames = {}
    loaded_files = []
    for file in files_to_load:
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        data_frames[file] = df
        loaded_files.append(file)
    print(f"총 {len(loaded_files)}개를 불러왔습니다 ({', '.join(loaded_files)})")
    
    return data_frames

In [35]:
def plot_daily_generation(data_frames, selected_dates=None):
    # 첫 번째 데이터를 기준으로 날짜 목록 가져오기
    first_df = list(data_frames.values())[0]
    dates_array = np.array(first_df['Date'].unique())
    
    if selected_dates is None:
        # 사용 가능한 날짜 목록 출력
        print("사용 가능한 날짜 목록:")
        for i, date in enumerate(dates_array):
            print(f"{i}: {date}")
        
        # 사용자 입력 받기
        print("\n플랏할 날짜 번호를 선택하세요 (여러 개인 경우 쉼표로 구분):")
        selected_indices = input().strip().split(',')
        selected_dates = dates_array[[int(idx.strip()) for idx in selected_indices]]
    else:
        # selected_dates가 정수인 경우 해당 인덱스의 날짜로 변환
        if isinstance(selected_dates, int):
            selected_dates = [dates_array[selected_dates]]
    
    # 각 선택된 날짜별로 플랏
    for date in selected_dates:
        plt.figure(figsize=(12, 6))
        
        # 각 발전기(CSV 파일)별로 플랏
        for file_name, df in data_frames.items():
            # 해당 날짜의 데이터만 선택
            mask = df['Date'] == date
            daily_hours = df[mask]['Hour'].to_numpy()
            daily_generation = df[mask]['Electricity Generated'].to_numpy()
            
            # 발전량 플랏
            label = f'DER {file_name.split(".")[0]}'
            plt.plot(daily_hours, daily_generation, label=label, marker='o')
        
        plt.title(f'Daily Generation - {date}')
        plt.xlabel('Hour')
        plt.ylabel('Electricity Generated (kW)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(False)
        plt.xticks(range(0, 24))
        plt.tight_layout()
        plt.show()

In [59]:
def plot_daily_generation(data_frames, selected_dates=None):
    # 첫 번째 데이터를 기준으로 날짜 목록 가져오기
    first_df = list(data_frames.values())[0]
    dates_array = np.array(first_df['Date'].unique())
    
    # selected_dates 처리
    if selected_dates is None:
        # None이면 모든 날짜 사용
        dates_to_plot = dates_array
    elif isinstance(selected_dates, (int, np.integer)):
        # 숫자 하나만 입력한 경우
        dates_to_plot = [dates_array[selected_dates]]
    elif isinstance(selected_dates, list):
        # 날짜 리스트를 입력한 경우
        dates_to_plot = [dates_array[i] for i in selected_dates]
    else:
        raise ValueError("selected_dates must be None, int, or list of ints")
    
    # 각 선택된 날짜별로 플랏
    for date in dates_to_plot:
        plt.figure(figsize=(12, 6))
        
        # 각 발전기(CSV 파일)별로 플랏
        for file_name, df in data_frames.items():
            # 해당 날짜의 데이터만 선택
            mask = df['Date'] == date
            daily_hours = df[mask]['Hour (Eastern Time, Daylight-Adjusted)'].to_numpy()
            daily_generation = df[mask]['Electricity Generated'].to_numpy()
            
            # 발전량 플랏
            label = f'DER {file_name.split(".")[0]}'
            plt.plot(daily_hours, daily_generation, label=label, marker='o')
        
        plt.title(f'Daily Generation - {date}')
        plt.xlabel('Hour')
        plt.ylabel('Electricity Generated (kW)')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(False)
        plt.xticks(range(0, 24))
        plt.yticks([])
        plt.tight_layout()
        plt.show()


In [ ]:
data = load_generation_data()
plot_daily_generation(data, [5, 14, 17, 18, 19, 24, 25, 29, 30, 31])


TypeError: 'int' object is not iterable